In [83]:
import requests
import requests_ftp
import requests_cache
import lxml
from lxml import html
from bs4 import BeautifulSoup
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd
import urllib
import json
import cloudscraper
import sqlalchemy as sqla

In [85]:
##Filtering Target Regions (Americas)
regiondata=pd.DataFrame(columns=['code','country'])

for i in range(70):
    regiondata.loc[i,"code"]=clim.find_all(name='li')[:70][i].get('data-value')
    regiondata.loc[i,"country"]=clim.find_all(name='li')[:70][i].text

pd.set_option("max_rows", None)
regiondata=regiondata.loc[[1,4,9,10,11,13,14,15,20,21,28,30,44,46,50,51,55,62,67]]
regiondata["region"]=["NA", "SA", "SA", "SA","NA", "SA" , "SA" , "NA" ,  "NA" , "SA" ,  "NA" , "NA" , "NA" , "NA" ,  "NA" , "SA" ,  "SA" ,  "NA" , "SA"]

regiondata=regiondata.reset_index()
pd.reset_option('^display.', silent=True)
regiondata

,index,code,country,region
0,1,us,United States,NA
1,4,ar,Argentina,SA
2,9,bo,Bolivia,SA
3,10,br,Brazil,SA
4,11,ca,Canada,NA
5,13,cl,Chile,SA
6,14,co,Colombia,SA
7,15,cr,Costa Rica,NA
8,20,do,Dominican Republic,NA
9,21,ec,Ecuador,SA


In [84]:
##Setting up sql engine and scraper for tests
engine = sqla.create_engine('sqlite:///top200data')
url = "https://spotifycharts.com/regional/global/daily/2021-05-25"
scraper = cloudscraper.create_scraper()
r = scraper.get("https://spotifycharts.com/regional/global/daily/2021-05-25")
clim = BeautifulSoup(r.text,'lxml')

In [86]:
##Test printing url loop
url = "https://spotifycharts.com/regional/{}/weekly/2021-05-14--2021-05-21".format(regiondata.loc[1,"code"])
url

'https://spotifycharts.com/regional/ar/weekly/2021-05-14--2021-05-21'

In [87]:
##Test Parsing Scraped Information
rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
link = clim.find_all(name='a')[6:-5] #only song links
streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
info[0].text.splitlines()[2][3:]
streams[0].text

'12,586,645'

In [89]:
info[0].text.splitlines()

['', 'good 4 u', 'by Olivia Rodrigo']

In [90]:
##Test Scrape for One Date, One Region for One Row
id = "date_{}_region".format(int(rank[0].text))
filler = (id, int(rank[0].text), info[0].text.splitlines()[1] , "date" , info[0].text.splitlines()[2][3:], "region")
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'PRIMARY KEY (id));')
engine.execute('INSERT INTO "top200" '
               '(id, rank, song, date, artist,region) '
               'VALUES (?,?,?,?,?,?) ', filler )
pd.read_sql('select * from top200',engine)

,id,rank,song,date,artist,region
0,date_1_region,1,good 4 u,date,Olivia Rodrigo,region


In [92]:
##Test Scrape for One Date and One Region for All Rows
import time
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date DATE, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')


for i in range(200):
    id = "date_{}_region".format(int(rank[i].text))
    filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], "region", link[i].get('href')[-22:],streams[i].text)

    engine.execute('INSERT INTO "top200" '
                   '(id, rank, song, date, artist,region, songcode, streams) '
                   'VALUES (?,?,?,?,?,?,?,?) ', filler )
print("--- %s seconds ---" % (time.time() - start_time))
pd.read_sql('select * from top200',engine)


--- 3.062173843383789 seconds ---


,id,rank,song,date,artist,region,songcode,streams
0,date_1_region,1,Chantaje (feat. Maluma),date,Shakira,region,6mICuAdrwEjh6Y6lroV2Kg,"1,478,972"
1,date_2_region,2,Reggaetón Lento (Bailemos),date,CNCO,region,3AEZUABDXNtecAOSC1qTfo,"1,305,366"
2,date_3_region,3,Vente Pa' Ca (feat. Maluma),date,Ricky Martin,region,7DM4BPaS7uofFul3ywMe46,"1,160,857"
3,date_4_region,4,Safari,date,"J Balvin, Pharrell Williams, BIA, Sky",region,6rQSrBHf7HlZjtcMZ4S4bO,"975,009"
4,date_5_region,5,Cuando Se Pone a Bailar,date,Rombai,region,1MpKZi1zTXpERKwxmOu1PH,"912,763"
...,...,...,...,...,...,...,...,...
195,date_196_region,196,Playa y Arena (Remix),date,"Mark B., Gabriel Pagan, Ozuna",region,4Q6Z7fn2xVhPZ1xn0XfEOY,"86,684"
196,date_197_region,197,Aventura,date,"Pijama Party, Valentina Olguin",region,1y9mv4NMheIuIUKf9hsIsy,"85,948"
197,date_198_region,198,Trátame Suavemente - Remasterizado 2007,date,Soda Stereo,region,65DBZofI0b79kfHTcWWDuU,"85,933"
198,date_199_region,199,Su Novio Anterior,date,"El Villano, J-One, Newyorkeeno",region,64CvAy6EPQpxCNbW2LNze9,"85,822"


In [78]:
##Test Scrape for One Date and All Regions
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'regioncode VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')

scraper = cloudscraper.create_scraper()
for place in range(19):


    url = "https://spotifycharts.com/regional/{}/weekly/2020-12-25--2021-01-01".format(regiondata.loc[place,'code'])

    r = scraper.get(url)
    clim = BeautifulSoup(r.text,'lxml')
    
    rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
    info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
    link = clim.find_all(name='a')[6:-5]  #only song links
    streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
    
    for i in range(len(rank)):
        id = "date_{}_{}".format(int(rank[i].text),place)
        filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], regiondata.loc[place,'code'], link[i].get('href')[-22:], streams[i].text)

        engine.execute('INSERT INTO "top200" '
                       '(id, rank, song, date, artist, regioncode, songcode, streams) '
                   'VALUES (?,?,?,?,?,?,?,?) ', filler )


print("--- %s seconds ---" % (time.time() - start_time))
#pd.read_sql('select * from top200',engine)


--- 86.46330976486206 seconds ---


In [81]:
pd.reset_option('^display.', silent=True)
pd.read_sql('select * from top200',engine)


,id,rank,song,date,artist,region,regioncode,songcode,streams
0,date_1_0,1,Rockin' Around The Christmas Tree,date,Brenda Lee,None,us,2EjXfH91m7f8HiJN1yQg97,"5,662,961"
1,date_2_0,2,34+35,date,Ariana Grande,None,us,6Im9k8u9iIzKMrmV7BWtlF,"5,425,062"
2,date_3_0,3,Jingle Bell Rock,date,Bobby Helms,None,us,7vQbuQcyTflfCIOu3Uzzya,"5,326,633"
3,date_4_0,4,Mood (feat. iann dior),date,24kGoldn,None,us,3tjFYV6RSFtuktYl3ZtYcq,"5,204,578"
4,date_5_0,5,"Lemonade (feat. Gunna, Don Toliver & NAV)",date,Internet Money,None,us,7hxHWCCAIIxFLCzvDgnQHX,"5,165,110"
...,...,...,...,...,...,...,...,...,...
3795,date_196_18,196,BOOKER T,date,Bad Bunny,None,uy,26w9NTiE9NGjW1ZvIOd1So,"13,961"
3796,date_197_18,197,Bajito No,date,Los Negroni,None,uy,2NbMHtHIp8jH4oglpEx0Jc,"13,873"
3797,date_198_18,198,Levitating (feat. DaBaby),date,Dua Lipa,None,uy,5nujrmhLynf4yMoMtj8AQF,"13,858"
3798,date_199_18,199,AYNEA REMIX,date,"FMK, Maria Becerra, Beret",None,uy,5dDxdj2MrqrYCGzqXfJycL,"13,844"


In [97]:
##Setting Targeted Dates for Scraping
import datetime
start_date = datetime.date(2017, 1, 6)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days = 7)
ls_date = []
while start_date <= end_date:
    ls_date.append(str(start_date))
    start_date += delta
ls_date[206]

'2020-12-18'

In [98]:
##Scrape for Dataset with All Dates and All Regions
import time
start_time = time.time()
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'regioncode VARCHAR, '
               'songcode VARCHAR, '
               'streams INTEGER NOT NULL, '
               'PRIMARY KEY (id));')

scraper = cloudscraper.create_scraper()

##Since there are too many rows, the data is scraped in different time periods, hence the for loop noi covering all dates
for k in range(206,208):
    for place in range(len(regiondata)):

        url = "https://spotifycharts.com/regional/{}/weekly/{}--{}".format(regiondata.loc[place,'code'], ls_date[k], ls_date[k+1])
        r = scraper.get(url)
        clim = BeautifulSoup(r.text,'lxml')
        rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
        info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
        link = clim.find_all(name='a')[6:-5] #only song links
        streams = clim.find_all(name='td',attrs={'class':'chart-table-streams'})
          

        for i in range(len(rank)):
            id = "{}_{}_{}".format(ls_date[k],int(rank[i].text),regiondata.loc[place,'code'])
            filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , ls_date[k] , info[i].text.splitlines()[2][3:], regiondata.loc[place,'code'], link[i].get('href')[-22:], streams[i].text)
            engine.execute('INSERT INTO "top200" '
                           '(id, rank, song, date, artist, regioncode, songcode, streams) '
                       'VALUES (?,?,?,?,?,?,?,?) ', filler )
    

print("--- %s seconds ---" % (time.time() - start_time))

# weeklytop200_df =  pd.read_sql('select * from top200',engine)

pd.read_sql('select * from top200',engine)

--- 165.77355027198792 seconds ---


,id,rank,song,date,artist,regioncode,songcode,streams
0,2020-12-18_1_us,1,Rockin' Around The Christmas Tree,2020-12-18,Brenda Lee,us,2EjXfH91m7f8HiJN1yQg97,"12,796,698"
1,2020-12-18_2_us,2,All I Want for Christmas Is You,2020-12-18,Mariah Carey,us,0bYg9bo50gSsH3LtXe2SQn,"11,853,295"
2,2020-12-18_3_us,3,Jingle Bell Rock,2020-12-18,Bobby Helms,us,7vQbuQcyTflfCIOu3Uzzya,"11,784,614"
3,2020-12-18_4_us,4,It's the Most Wonderful Time of the Year,2020-12-18,Andy Williams,us,1IcR6RlgvDczfvoWJSSY2A,"11,067,032"
4,2020-12-18_5_us,5,Last Christmas,2020-12-18,Wham!,us,2FRnf9qhLbvw8fu4IBXx78,"10,366,103"
...,...,...,...,...,...,...,...,...
7595,2020-12-25_196_uy,196,BOOKER T,2020-12-25,Bad Bunny,uy,26w9NTiE9NGjW1ZvIOd1So,"13,961"
7596,2020-12-25_197_uy,197,Bajito No,2020-12-25,Los Negroni,uy,2NbMHtHIp8jH4oglpEx0Jc,"13,873"
7597,2020-12-25_198_uy,198,Levitating (feat. DaBaby),2020-12-25,Dua Lipa,uy,5nujrmhLynf4yMoMtj8AQF,"13,858"
7598,2020-12-25_199_uy,199,AYNEA REMIX,2020-12-25,"FMK, Maria Becerra, Beret",uy,5dDxdj2MrqrYCGzqXfJycL,"13,844"


In [99]:
##Turning Scraped Subsets into CSV Files
#pd.read_sql('SELECT * FROM top200 WHERE "id" LIKE "%_1_%"; ', engine)
pd.read_sql('select * from top200',engine).to_csv("top200_201218--201225.csv")

In [120]:
pd.read_csv('top200_170106--180330.csv')

,Unnamed: 0,id,rank,song,date,artist,regioncode,songcode,streams
0,0,2017-01-06_1_us,1,Shape of You,2017-01-06,Ed Sheeran,us,7qiZfU4dY1lWllzX7mPBI3,"10,969,080"
1,1,2017-01-06_2_us,2,Bad and Boujee (feat. Lil Uzi Vert),2017-01-06,Migos,us,4Km5HrUvYTaSUfiSGPJeQR,"10,112,877"
2,2,2017-01-06_3_us,3,Fake Love,2017-01-06,Drake,us,343YBumqHu19cGoGARUTsd,"8,459,802"
3,3,2017-01-06_4_us,4,Castle on the Hill,2017-01-06,Ed Sheeran,us,6PCUP3dWmTjcTtXY02oFdT,"7,600,859"
4,4,2017-01-06_5_us,5,Starboy,2017-01-06,"The Weeknd, Daft Punk",us,5aAx2yezTd8zXrkmtKl66Z,"7,245,714"
...,...,...,...,...,...,...,...,...,...
246795,246795,2018-03-30_196_sv,196,Friends (with BloodPop®),2018-03-30,Justin Bieber,sv,7nZmah2llfvLDiUjm0kiyz,"4,966"
246796,246796,2018-03-30_197_sv,197,In the Name of Love,2018-03-30,"Martin Garrix, Bebe Rexha",sv,04DwTuZ2VBdJCCC5TROn7L,"4,919"
246797,246797,2018-03-30_198_sv,198,Devuélveme a mi chica,2018-03-30,Hombres G,sv,1Wrzhfa5bNlqvsnCztz190,"4,893"
246798,246798,2018-03-30_199_sv,199,"Let Me Go (with Alesso, Florida Georgia Line &...",2018-03-30,Hailee Steinfeld,sv,5AEtlRudpgdT5FtNiuly6Y,"4,887"


In [130]:
##Merging All Subsets into One Dataframe
pd.concat([pd.read_csv('top200_170106--180330.csv'), 
           pd.read_csv('top200_180330--190503.csv'), 
           pd.read_csv('top200_190503--200522.csv'), 
           pd.read_csv('top200_200522--201218.csv'), 
           pd.read_csv('top200_201218--201225.csv')])\
.drop(columns=['Unnamed: 0']).reset_index(drop=True).drop_duplicates().reset_index(drop=True).to_sql("top200data", engine)


In [131]:
##Export Full Dataset
pd.read_sql('select * from top200data',engine).to_csv("top200data.csv")

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [18]:
uniquesongs=pd.read_sql('SELECT DISTINCT song,songcode FROM top200',engine)
uniquesongs

,song,songcode
0,good 4 u,4ZtFanR9U6ndgddUvNcjcG
1,Butter,3VqeTFIvhxu3DIe4eZVzGq
2,deja vu,6HU7h9RYOaPRFeh0R3UeAr
3,MONTERO (Call Me By Your Name),67BtfxlNbhBmCDR2L2l8qd
4,Kiss Me More (feat. SZA),748mdHapucXQri7IAO8yFK
...,...,...
195,Savage Love (Laxed - Siren Beat),1xQ6trAsedVPCdbtDAmk0c
196,La Nota,5VzNsUnLmYcRfRfxpQsusa
197,If the World Was Ending - feat. Julia Michaels,2kJwzbxV2ppxnQoYw4GLBZ
198,Somebody That I Used To Know,1qDrWA6lyx8cLECdZE7TV7


In [78]:
results = spotify.search(q='artist:' + 'Doja Cat', type='artist')
results['artists']['items'][0]['genres']
# items = results['artists']['items']
# items['items']

['dance pop', 'pop', 'pop rap']

In [76]:
results['artists']['items'][0]['genres']

['dance pop', 'pop', 'pop rap']

In [27]:
url="https://www.chosic.com/music-genre-finder"
r=requests.get(url, params={'track':'4ZtFanR9U6ndgddUvNcjcG'})
glimhtml = r.text
glim = BeautifulSoup(glimhtml,'lxml')
#glim.findall(name = 'div', attrs = {'class':'spotify_result'})


In [81]:
artistlist = pd.DataFrame(columns=['artist','genre'])
artistlist['artist'] = pd.read_sql('SELECT DISTINCT artist FROM test200 WHERE artist IS NOT NULL',engine)['artist'].apply(lambda x: x.split(', ')[0]).unique()
artistlist
for i in range(1189):
    results = spotify.search(q='artist:' + artistlist.loc[i,'artist'], type='artist')
    artistlist.loc[i,'genre']=results['artists']['items'][0]['genres']
artistlist

IndexError: list index out of range

In [8]:
spotify.audio_features('4ZtFanR9U6ndgddUvNcjcG')[0]['danceability']

0.563

In [14]:
pd.read_csv('top200_170106--180330.csv').to_sql("test200",engine)

In [52]:
pd.read_sql('SELECT DISTINCT artist, id, songcode  FROM test200',engine)[pd.read_sql('SELECT DISTINCT artist, id, songcode  FROM test200',engine)['artist'].isnull()]

,artist,id,songcode
1662,None,2017-01-06_63_do,5d3cgVLdEPWUmyAFIJwDrk
5467,None,2017-01-13_68_do,5d3cgVLdEPWUmyAFIJwDrk
9270,None,2017-01-20_71_do,5d3cgVLdEPWUmyAFIJwDrk
13076,None,2017-01-27_77_do,5d3cgVLdEPWUmyAFIJwDrk
16884,None,2017-02-03_85_do,5d3cgVLdEPWUmyAFIJwDrk
...,...,...,...
166601,None,2017-11-03_2_py,1YqcGlCHNquxBhlUZsjhMT
166801,None,2017-11-03_2_sv,1YqcGlCHNquxBhlUZsjhMT
167001,None,2017-11-03_2_uy,1YqcGlCHNquxBhlUZsjhMT
198377,None,2018-01-05_178_br,5WrEqIllVjKgdmAmbhmKcY
